# Dependencies

In [ ]:
# Install necessary packages
%pip install --quiet --upgrade diffusers transformers accelerate invisible_watermark mediapy


In [ ]:
# Flag to determine whether to use the refiner model
use_refiner = False

# Model Loading

In [ ]:
# Import required libraries
import torch
import mediapy as media
import random
import sys

# Import the DiffusionPipeline from diffusers
from diffusers import DiffusionPipeline

# Initialize the DiffusionPipeline with a pre-trained base model
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)

# Check if using refiner
if use_refiner:
    # Initialize a DiffusionPipeline with a pre-trained refiner model
    refiner = DiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-refiner-1.0",
        text_encoder_2=pipe.text_encoder_2,
        vae=pipe.vae,
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16",
    )

    # Move the refiner model to CUDA (GPU)
    refiner = refiner.to("cuda")

    # Enable model CPU offload for the base pipeline
    pipe.enable_model_cpu_offload()
else:
    # Move the base pipeline to CUDA (GPU)
    pipe = pipe.to("cuda")

In [ ]:
# Specify a prompt for image generation
#prompt = "abstract colors, texture, film grain:0. 1 intricate close up dramatic portrait of a (cyberpunk)1. 2 porsche standing on a rainy bustling crowded cyberpunk city street, ultra realistic texture, foggy morning, cinematic movie still frame, gtr, altered carbon, (analog style)1. 3 (film grain)1. 3, stephan martiniere Wadim Kashin Simon Stalenhag michal karcz ismail inceoglu"
prompt = "an outdoor sculpture of a head using discarded car parts, highlighting it's beauty, highly detailed, 8k"
# Generate a random seed
seed = random.randint(0, sys.maxsize)

# Use the pipeline to generate images based on the prompt
images = pipe(
    prompt=prompt,
    output_type="latent" if use_refiner else "pil",
    generator=torch.Generator("cuda").manual_seed(seed),
).images

# If using the refiner, refine the images
if use_refiner:
    images = refiner(
        prompt=prompt,
        image=images,
    ).images

# Print the prompt and seed
print(f"Prompt:\t{prompt}\nSeed:\t{seed}")

# Display the generated images
media.show_images(images)

# Save the first image as "output.jpg"
images[0].save("output.jpg")
